# Train the model

In [ ]:
import os
from datetime import datetime
import boto3
from ultralytics import YOLO

In [ ]:
!tar -xzf dataset-full.tar.gz

In [ ]:
EPOCHS = int(os.environ.get("epochs", 50))
IMGSZ = 640
NB_FROZEN_LAYER = 0
DATA_CONFIG_PATH = "utils/data.yaml"

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")

In [ ]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [ ]:
client.download_file(AWS_S3_BUCKET, "base_model.pt", "base-model.pt")

## Load the base model and tune the model

In [ ]:
model = YOLO('base-model.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ, freeze=NB_FROZEN_LAYER, workers=0)

## Export and Upload the Model

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
model.export(format="onnx")

In [ ]:
model_path = "runs/detect/train/weights/best.onnx"

version = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
s3_dir = f"models/v{version}"
s3_dir_latest = "models/latest"
s3_onnx_path = f"{s3_dir}/model.onnx"
s3_onnx_path_latest = f"{s3_dir_latest}/model.onnx"

client.upload_file(model_path, AWS_S3_BUCKET, s3_onnx_path)
print(f"Model uploaded to S3 at {s3_onnx_path}")
client.upload_file(model_path, AWS_S3_BUCKET, s3_onnx_path_latest)
print(f"Model uploaded to S3 at {s3_onnx_path_latest}")

client.upload_file('runs/detect/train/weights/best.pt', AWS_S3_BUCKET, f"{s3_dir}/model.pt")
client.upload_file('runs/detect/train/weights/best.pt', AWS_S3_BUCKET, f"{s3_dir_latest}/model.pt")
client.upload_file('runs/detect/train/results.csv', AWS_S3_BUCKET, f"{s3_dir}/results.csv")
client.upload_file('runs/detect/train/results.csv', AWS_S3_BUCKET, f"{s3_dir_latest}/results.csv")